In [ ]:
!pip install selenium webdriver-manager

In [7]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium.common.exceptions import UnexpectedAlertPresentException
import re
import time



options = Options()
options.add_experimental_option("detach", True) # 실행 후 창 유지
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

try:
    # 1. 사이트 접속 (누락되었던 주소 추가)    
    driver.get("https://gcloud.csi.go.kr/cmq/main.do")
    print("\n" + "="*60)
    print(" [작업 지시]")
    print(" 1. 크롬 창에서 로그인을 완료하세요.")
    print(" 2. 상세 페이지의 '공사개요'가 화면에 보이면 Enter를 누르세요.")
    print("="*60)
    input(">> 준비 완료 후 Enter를 누르세요: ")

    # 2. 하단 데이터 활성화를 위한 강제 스크롤 및 대기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2) 

    print("\n" + "="*60)
    print(" [데이터 추출 결과]")
    print("="*60)

    # --- [2번 영역: 이미 알고 있는 속성 ID로 강제 추출] ---
    print("\n[2. 공사개요 상세]")
    # bizNm, addr, bgngDt 등 알고 계신 속성값을 직접 타겟팅합니다.
    construction_ids = {
        "공사명": "bizNm",
        "현장소재지": "addr",
        "착공일": "bgngDt",
        "준공예정일": "cmplPlanDt"
    }

    for label, eid in construction_ids.items():
        try:
            element = driver.find_element(By.ID, eid)
            # 2번 항목 해결의 핵심: innerText, value, text를 모두 확인
            val = element.get_attribute('innerText').strip() or \
                  element.get_attribute('value') or \
                  element.text.strip()
            
            print(f"> {label} : {val if val else '데이터 없음'}")
        except:
            print(f"> {label} : 요소를 찾을 수 없음 (ID: {eid})")

    # --- [3번 영역: 잘 나온다고 하신 기존 로직] ---
    print("\n[3. 기타 항목]")
    other_labels = ["성과 이용 목적", "발주자", "시공자", "국가중요시설 여부"]
    for label in other_labels:
        try:
            val = driver.find_element(By.XPATH, f"//th[contains(text(), '{label}')]/following-sibling::td").text.strip()
            print(f"> {label} : {val}")
        except:
            continue

    print("\n" + "="*60)
    print(" 추출 작업이 완료되었습니다.")

except Exception as e:
    print(f"\n[오류 발생] {e}")




 [작업 지시]
 1. 크롬 창에서 로그인을 완료하세요.
 2. 상세 페이지의 '공사개요'가 화면에 보이면 Enter를 누르세요.

 [데이터 추출 결과]

[2. 공사개요 상세]
> 공사명 : 부천대장 공종주택지구 조성공사 2공구
> 현장소재지 : 경기도 부천시 오정구 벌말로 43
> 착공일 : 2024-04-25
> 준공예정일 : 요소를 찾을 수 없음 (ID: cmplPlanDt)

[3. 기타 항목]
> 성과 이용 목적 : 품질시험·검사 대행
> 발주자 : 한국토지주택공사 인천지역본부 (담당자: 최기찬)
> 시공자 : 현대엔지니어링 주식회사
> 국가중요시설 여부 : 해당없음

 추출 작업이 완료되었습니다.
